In [2]:
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm
import ast, re
from time import sleep
from bs4 import BeautifulSoup
data_folder = ''
api = ''
import warnings
warnings.filterwarnings('ignore')

## Extract "Undesirable Effects" Section

### Extract from raw elements

In [ ]:
drug_df = pd.read_csv(data_folder+'data/final_UK/drug_data.csv')
print(drug_df.shape, drug_df.product_id.nunique(), drug_df.ingredient_id.nunique())
drug_df.head(1)

(11450, 6) 9334 1978


,ingredient_id,ingredient_name,product_id,product_name,active_ingredients,company_name
0,/emc/ingredient/1524,"2,4-dichlorobenzyl alcohol",/emc/product/1589/smpc,\n Boots Antiseptic Cream\n...,"allantoin, cetrimide, 2,4-dichlorobenzyl alcohol",THE BOOTS COMPANY PLC


In [ ]:
drug_ade_text = []
for product in tqdm(drug_df.product_id.unique().tolist()):
  f = (data_folder+'data/raw/{}.txt'.format(product.split('/')[-2]))
  with open(f) as fi:
      s = BeautifulSoup(fi, 'html.parser')
  for i in s.find_all('details'):
      if '4.8 Undesirable effects' in i.text:
        ade_text = i
        drug_ade_text.append([product.split('/')[-2], ade_text])
drug_ade_df = pd.DataFrame(drug_ade_text, columns = ['product_id', 'drug_text'])
drug_ade_df.to_csv(data_folder+'data/drug_ade_data_raw.csv', index=False)

In [ ]:
drug_ade_text = []
for product in tqdm(drug_df.product_id.unique().tolist()):
  f = (data_folder+'data/raw/{}.txt'.format(product.split('/')[-2]))
  with open(f) as fi:
      s = BeautifulSoup(fi, 'html.parser')
  for i in s.find_all('details'):
      text_low = i.text.lower()
      if '4.4 special warnings and precautions for use' in text_low:
        ade_text = i
        drug_ade_text.append([product.split('/')[-2], ade_text])
drug_ade_df = pd.DataFrame(drug_ade_text, columns = ['product_id', 'drug_text'])
drug_ade_df.to_csv(data_folder+'data/drug_wp_data_raw.csv', index=False)

100%|██████████| 9334/9334 [45:39<00:00,  3.41it/s]


In [ ]:
drug_ade_text = []
for product in tqdm(drug_df.product_id.unique().tolist()):
  f = (data_folder+'data/raw/{}.txt'.format(product.split('/')[-2]))
  with open(f) as fi:
      s = BeautifulSoup(fi, 'html.parser')
  for i in s.find_all('details'):
      text_low = i.text.lower()
      if '4.3 contraindications' in text_low:
        ade_text = i
        drug_ade_text.append([product.split('/')[-2], ade_text])
drug_ade_df = pd.DataFrame(drug_ade_text, columns = ['product_id', 'drug_text'])
drug_ade_df.to_csv(data_folder+'data/drug_bw_data_raw.csv', index=False)

100%|██████████| 9334/9334 [11:27<00:00, 13.57it/s]


---

In [ ]:
drug_ade_df = pd.read_csv(data_folder+'data/drug_ade_data_raw.csv')
print(drug_ade_df.shape)
drug_ade_df.head()

(9222, 2)


,product_id,drug_text
0,1589,"<details><summary data-evt=""smpcSectionOpen"" i..."
1,4223,"<details><summary data-evt=""smpcSectionOpen"" i..."
2,5606,"<details><summary data-evt=""smpcSectionOpen"" i..."
3,2700,"<details><summary data-evt=""smpcSectionOpen"" i..."
4,625,"<details><summary data-evt=""smpcSectionOpen"" i..."


In [ ]:
all_drugs = [int(i.split('/')[-2]) for i in drug_df['product_id'].unique().tolist()]
parsed_drugs = drug_ade_df.product_id.unique().tolist()
missing_drugs = list(set(all_drugs) - set(parsed_drugs))
for drug in tqdm(missing_drugs):
  f = (data_folder+'data/raw/{}.txt'.format(str(drug)))
  with open(f) as fi:
      s = BeautifulSoup(fi, 'html.parser')
  if 'undesirable effects' in s.text.lower():
    print(drug)

100%|██████████| 316/316 [00:59<00:00,  5.27it/s]


---
### Extract Individual ADEs from Section Tables
Frequencies :
- Very common : >=10%
- Common : >=1% and <10%
- Uncommon : >=0.1% and <1%
- Rare >0.01% and <0.1%
- Very Rare <0.01%

In [ ]:
drug_ade_df = pd.read_csv(data_folder+'data/drug_ade_data_raw.csv')
print(drug_ade_df.shape)
drug_ade_df.head(1)

(9222, 2)


,product_id,drug_text
0,1589,"<details><summary data-evt=""smpcSectionOpen"" i..."


In [ ]:
big_table_list = []
for p_id, i in tqdm(zip(drug_ade_df.product_id.tolist(), drug_ade_df.drug_text.tolist())):
  i = BeautifulSoup(i, 'html.parser')
  if len(i.find_all('table')) > 0:
    table = i.find('table')
    rows = table.find_all('tr')
    for row in rows:
      big_table_list.append([p_id, row])

9222it [02:08, 71.67it/s]


In [ ]:
freqs = ['very common', 'common', 'uncommon', 'rare', 'very rare', 'not known']
socs =  ['blood and lymphatic system disorders','cardiac disorders', 'congenital, familial and genetic disorders',
         'ear and labyrinth disorders', 'endocrine disorders', 'eye disorders', 'gastrointestinal disorders',
         'general disorders and administration site conditions', 'hepatobiliary disorders', 'immune system disorders',
         'infections and infestations', 'injury, poisoning and procedural complications', 'investigations', 'metabolism and nutrition disorders',
         'musculoskeletal and connective tissue disorders', 'neoplasms benign, malignant and unspecified (incl cysts and polyps)',
         'nervous system disorders', 'pregnancy, puerperium and perinatal conditions', 'psychiatric disorders',
         'renal and urinary disorders', 'reproductive system and breast disorders', 'respiratory, thoracic and mediastinal disorders',
         'skin and subcutaneous tissue disorders', 'social circumstances', 'surgical and medical procedures', 'vascular disorders', 'product issues']
titles = ['system organ class', 'frequency', 'adverse events']

In [ ]:
processed_list = []
for row in tqdm(big_table_list):
  drug = row[0]
  r = row[1]
  items = [i.text.replace('\n','').lower() for i in r.find_all('td')]
  if any([True for e in titles if e in items]):
    continue
  else:
    f, s, a = None, None, None
    for i in items:
      i = i.strip().replace('*','')
      if i in freqs: f = i
      elif i in socs: s = i
      else: a = i
    processed_list.append([drug, f, s, a])

100%|██████████| 158931/158931 [00:05<00:00, 31269.74it/s]


In [ ]:
processed_df = pd.DataFrame(processed_list, columns = ['product_id', 'freq', 'soc', 'ade'])
processed_df['freq'] = processed_df.apply(lambda x: str(x.ade).split(':')[0] if str(x.ade) !='nan' and str(x.ade).split(':')[0] in freqs else x.freq , axis = 1)
processed_df.to_csv(data_folder+'data/drug_ade_data_parsed.csv', index=False)
processed_df.head()

,product_id,freq,soc,ade
0,4223,not known,immune system disorders,hypersensitivityab1
1,4223,not known,gastrointestinal disorders,"glossodyniaab, oral discomfortab"
2,5606,not known,immune system disorders,hypersensitivityab1
3,5606,not known,gastrointestinal disorders,"glossodyniaab, oral discomfortab"
4,2700,not known,immune system disorders,hypersensitivityab1


### Map Terms to MedDRA codes (section tables)

In [9]:
processed_df = pd.read_csv(data_folder+'data/drug_ade_data_parsed.csv')
processed_df = processed_df[['product_id', 'freq', 'soc', 'ade']]
processed_df.head()

,product_id,freq,soc,ade
0,4223,not known,immune system disorders,hypersensitivityab1
1,4223,not known,gastrointestinal disorders,"glossodyniaab, oral discomfortab"
2,5606,not known,immune system disorders,hypersensitivityab1
3,5606,not known,gastrointestinal disorders,"glossodyniaab, oral discomfortab"
4,2700,not known,immune system disorders,hypersensitivityab1


In [10]:
unique_terms = processed_df[['ade']].drop_duplicates()
unique_terms = unique_terms[unique_terms.ade.notna()]
unique_terms.head()

,ade
0,hypersensitivityab1
1,"glossodyniaab, oral discomfortab"
16,adverse reactions involving the treatment site...
17,"erythema, exfoliation, irritation, pain, pruri..."
18,"bleeding, desquamation, discharge, discomfort,..."


In [11]:
f = '/content/drive/MyDrive/pop_pharmacogenomics/onsides_intl/'
meddra_df = pd.read_csv(f+'external_data/umls_meddra_en.csv')
meddra_df['STR'] = meddra_df.STR.apply(lambda x: x.lower())
meddra_df['len'] = meddra_df.STR.apply(lambda x: len(x))
print(meddra_df.shape[0])
meddra_df = meddra_df[(meddra_df['len'] >= 5)]
meddra_df = meddra_df[(meddra_df.TTY.isin(['PT', 'LLT']))]
print(meddra_df.shape[0])
meddra_dict = dict(zip(meddra_df.STR, meddra_df.SDUI))
meddra_df.head(1)

116807
100897


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,len
0,C0000727,ENG,P,L0000727,VCW,S0584932,N,A0639292,NaN,NaN,10000647,MDR,PT,10000647,acute abdomen,3,N,256.0,13


In [12]:
found_ades = []
meddra_names = meddra_df.STR.tolist()
for ade_text in tqdm(unique_terms.ade.tolist()):
  ar_text = ' '.join(ade_text.split()).lower()
  found_terms = []
  for concept_name in meddra_names:
    if ar_text.find(concept_name) == -1:
      continue
    else:
      found_terms.append(concept_name)
  found_ades.append(found_terms)

100%|██████████| 22912/22912 [10:39<00:00, 35.81it/s]


In [13]:
unique_terms['exact_match_list'] = found_ades
unique_terms['exact_match_list'] = unique_terms['exact_match_list'].apply(lambda x: list(set(x)))
unique_terms.to_csv(data_folder+'drug_ade_data_parsed_text_unique.csv', index=False)
unique_terms.head()

,ade,exact_match_list
0,hypersensitivityab1,[hypersensitivity]
1,"glossodyniaab, oral discomfortab","[discomfort, glossodynia, oral discomfort]"
16,adverse reactions involving the treatment site...,"[local reaction, adverse reaction]"
17,"erythema, exfoliation, irritation, pain, pruri...","[exfoliation, erythema, pruritus]"
18,"bleeding, desquamation, discharge, discomfort,...","[discomfort, vesicle, oedema, vesicles, swelli..."


In [14]:
processed_df = processed_df.merge(unique_terms, on = 'ade', how = 'left')
processed_df['matched_codes'] = processed_df.exact_match_list.apply(lambda x: [meddra_dict[i] for i in x] if str(x) != 'nan' else None)
processed_df.to_csv(data_folder+'drug_ade_data_parsed.csv', index=False)
processed_df.head(1)

,product_id,freq,soc,ade,exact_match_list,matched_codes
0,4223,not known,immune system disorders,hypersensitivityab1,[hypersensitivity],[10020751]


### Extract Individual ADEs from Section Text

In [ ]:
drug_ade_df = pd.read_csv(data_folder+'data/drug_ade_data_raw.csv')
print(drug_ade_df.shape)
drug_ade_df.head(1)

(9222, 2)


,product_id,drug_text
0,1589,"<details><summary data-evt=""smpcSectionOpen"" i..."


In [ ]:
text_list = []
for p_id, i in tqdm(zip(drug_ade_df.product_id.tolist(), drug_ade_df.drug_text.tolist())):
  i = BeautifulSoup(i, 'html.parser')
  if len(i.find_all('table')) > 0:
    i.find('table').decompose() #remove the tables
  text_list.append([p_id, i.text])

9222it [01:47, 85.82it/s]


In [ ]:
processed_text_df = pd.DataFrame(text_list, columns = ['product_id', 'text'])
processed_text_df['text'] = processed_text_df['text'].apply(lambda x: str(x).lower().split('reporting of suspected adverse reactions')[0].split('4.8 undesirable effects')[-1].replace('\n',''))
processed_text_df.to_csv(data_folder+'drug_ade_text_parsed.csv', index=False)
processed_text_df.head()

,product_id,text
0,1589,hypersensitivity reactions may occasionally oc...
1,4223,the list of the following adverse effects rela...
2,5606,the list of the following adverse effects rela...
3,2700,the list of the following adverse effects rela...
4,625,the list of the following adverse effects rela...


###Use OnSIDES method to extract ADE from text

In [ ]:
#this has now been moved to 2.term_extraction_ade.ipynb

---
## Special Populations

this has been moved to the special_populations_extractions.ipynb notebook